# Setting up JupyterJSC

**Content creators**: Fabian Emmerich, Kristian Ehlert

In this first tutorial, you will be doing first steps on **JURECA**.
This tutorial assumes that you are familiar with the command prompt to a minimum degree.

JURECA has two different types of nodes: 

- Login Nodes: The entry point to the system.
  - Users here log in to manage their workspaces, move data, and submit jobs that are supposed to be run on the cluster.
  - Login nodes are not designed for computational workloads!
- Compute Nodes: The compute power of the system.
  - Each node has multiple CPUs (64), a large amount of RAM (512GB).
  - JURECA is a system that was especially designed for usage of GPUs and thus is equiped with 4 NVidia A100 GPUs (4 x 40GB vRAM) per node.
  - Compute nodes are detached from the internet.
  - JURECA in total has 200 nodes.

For detailed overviews of each system see [here](https://apps.fz-juelich.de/jsc/hps/jureca/configuration.html).

## Create a JupyterLab and select the required Jupyter kernel

For most of the bootcamp, we will work in Jupyter notebooks that have access to the JURECA cluster via [Jupter-JSC](https://jupyter-jsc.fz-juelich.de/hub/home).
Notebook in Jupyter-JSC (like all others) rely on so-called kernels.
A Jupyter kernel provides a software environment to run your notebooks in.
For Python, a kernel may provide a set of pre-installed packages.
However, it is also possible to use kernels with other software or even for other programming languages that allow interactive computing.

To get started right away, a kernel that specifies all required dependencies has been prepared for you.
It relies on an Apptainer image (similar to Docker) that is loaded in the background.

> **Note:** We will use Apptainer images as a Jupyter kernel.
> However, using Apptainer images is **not** enabled by default. 
> To be able to run Apptainer images, you will have to accept the user agreement for Apptainer images on JuDoor.
> To do so
> 
> 1. Login to [judoor](https://judoor.fz-juelich.de/login).
> 2. Go to: "Software" -> "+ Request access to restricted software" -> Select "Container Runtime Engine" -> "Send Request".

### Running a JupterLab

Now create a JupyterLab that runs on a queue of the HPC cluster

1. Login to [jupyter-jsc](https://jupyter-jsc.fz-juelich.de/hub/login).
2. Create a new JupyterLab, i.e. "NEW JUPYTERLAB":
Four different configurations tab will appear ("Lab Config", "Resources", "Kernels and Exensions").
  - "Lab Config": This defines the general configuration of the JupyperLab
    - Set "Name" for this JupyterLab.
    - Select newest "Version" (should be pre-selected).
    - Set "System" to JURECA, which is the sub-cluster used for this bootcamp.
    - Choose "Partition" reserved for the bootcamp "dc-gpu".
  - "Resources": This defines computing recourses allocated to this Jupyter Lab.
    - Set "Nodes" (cluster of tightly coupled processing units including GPUs) to 1
    - Set "Runtime" to the time you will approximately work on this notebook. You can also pick the maximum but just do not forget to shutdown the JupyterLab when not required.
  - "Kernels and Exensions": This defines the environments that are displayed. As we use our custom Jupyter kernel, this section can be ignored.
3. Launch your configured JupyterLab

Once the JuypterLab is ready, you will be connected to it. On the landing page you will see all applications that are accessible from the JupyerLab.

- In the Notebook section, you can launch interactive notebooks, e.g. for Python.
- The Console section allows you to launch interactive consoles such as an interactive Python session.
- On the bottom, in the Other section, you are able to launch a terminal from which you can interact with the shell of the system, e.g. to browse the file system, move files, or the like.

### Selecting the Jupter Kernel

To complete the following tutorials, you will use a Jupyter kernel which has all required packages installed. 1. Launch a JupyerLab on a login node via JupyterJSC and launch a terminal. Continue this notebook from there.

1. To create a custom kernel, you have to create a config file (`kernel.json`) in a directory in the `~/.local/share/jupyter/kernels` path, which Jupyter scans for custom configurations.
   1. Create a folder namend `maelstrom-bootcamp` in the required path

In [ ]:
!mkdir -p ~/.local/share/jupyter/kernels/maelstrom-bootcamp

B. Create a `kernel.json` file in the previously created path by executing the below cell. The `%%file <file path>` magic command at the top of the cell will write the content of the respective cell to the given path (file).

In [ ]:
%%file ~/.local/share/jupyter/kernels/maelstrom-bootcamp/kernel.json
{
 "argv": [
   "apptainer",
   "exec",
   "--cleanenv",
   "-B /usr/:/host/usr/,/etc/slurm:/etc/slurm,/usr/lib64:/host/usr/lib64,/opt/parastation:/opt/parastation,/usr/lib64/slurm:/usr/lib64/slurm,/opt/jsc:/opt/jsc",
   "--nv",
   "/p/project/training2330/a6/jupyter-kernel.sif",
   "python",
   "-m",
   "ipykernel",
   "-f",
   "{connection_file}"
 ],
 "language": "python",
 "display_name": "maelstrom-bootcamp-a6"
}

The `display_name` field in the above JSON structure will be the name under which the kernel will appear in the JupyterLab. 

The `argv` array contains the command that will be executed when the kernel is loaded by Jupyter. Here, we run a command inside an Apptainer image (`apptainer exec [...] jupyter-kernel.sif`) that launches a file with an ipykernel (`python -m ipykernel`). The other arguments passed to the command will allow us to use the software of the system (e.g. Slurm) from within the kernel.

Apptainer, formerly known as Singularity, is a container runtime that was designed for usage on high-performance systems. Containers in general enable to create software environments that can be run on any host, but are separated from the host's operating system. In general, such containers allow installing any software and run it on any system without requiring prerequisites - except Apptainer, of course. As a consequence, they provide the maximum amount of reproducibility. The Apptainer image we use here basically provides a Python environment with a set of packages that you will need to complete the rest of the tutorials.

It may take a minute or two for JupyterJSC to update the available kernels.
You can directly check whether the kernel is available via `jupyter kernelspec list`, which should return a list of all available kernels and their paths. This list should include a kernel with the path from above (`/p/home/jusers/<user>/juwels/.local/share/jupyter/kernels/maelstrom-bootcamp`).

You can execute this command directly from this notebook (see below cell) by using the magic command `!`, which executes the given command in the underlying shell (terminal) of the system (e.g. bash).

In [ ]:
!jupyter kernelspec list

1. Now, in the top menu bar, navigate to `Kernel > Change Kernel...`
   <img src="images/jupyterlab-kernel.png" width="60%" height="60%">
1. From the popup's dropdown, select the kernel `maelstrom-bootcamp-a6`.
1. Once the kernel is loaded, you will see it on the top right of the notebook.

   <img src="./images/jupyterlab-kernel-status.png" width="60%" height="60%">
   
   Hovering over the circle to the right of the name will show you the status of the kernel.
   Clicking on the field that contains the name of the kernel also allows you to switch the kernel as in step 1.

## Explore the supercomputer

You may have noticed that in the Options tab, by default `Partition > LoginNode` is selected.
In fact, JupyterJSC also allows you to launch a JupyterLab on login nodes without any time limit.
You can use these to perform regular tasks on the system (e.g. via terminal) or test simple Python routines.
But remember: the login nodes are not designed for heavy computations!


Now, open a terminal from the "Launcher" tab ("Other" > "Terminal") and use the command `squeue` to check the status of your job. Use

```bash
squeue
```

to inspect the current status of the queues. Enter

```bash
squeue -u <username>
```

to filter out only the lines of `squeue` that contain entries belonging to you user.

1. Using `sacct` and `squeue -u $USER` you will see your currently running interactive job that runs your JupyterLab.
2. Launch a terminal and figure out where you are located on the file system, e.g. via `ls` and `pwd`. Explore the system storage a bit. Take a look at the following paths:
   - /p
   - /p/project and /p/project/training2330/
   - /p/home/jusers and /p/home/jusers/$USER/
   - /p/scratch and /p/scratch/training2330
   - /p/project/training2330/a6

Open a terminal from within JupyterJSC and setup the code required for the bootcamp
Note, use the following commands in a terminal not in the notebook directly as `USER` does not seem to be available here.

1. Create a personal directory named like your user in the project folder located in /p/project/training2330/.

   ```bash
   mkdir /p/project/training2330/${USER}
   ```

2. Navigate to the project folder.

   ```bash
   cd /p/project/training2330/${USER}
   ```

3. Clone the [course material](https://gitlab.jsc.fz-juelich.de/esde/training/maelstrom_bootcamp) Git reposity to that folder.

   ```bash
   git clone https://gitlab.jsc.fz-juelich.de/esde/training/maelstrom_bootcamp.git
   ```

   On the left sidebar navigate to your newly created directory.
   Now, navigate to `File > New Launcher` in the top left corner and launch a Jupyter Notebook with kernel `maelstrom-bootcamp-a6` console.
   Execute a set of easy commands, e.g. print a simple statement.

4. **CRUCIAL**: Launch a Python notebook with the 'maelstrom-bootcamp-a6' kernel and `import torch` as a test.
   All future exercises are based on this environment so you need to be able to use this kernel.

If you are used to working with jupyter notebooks then the working environment will be very familiar to you. In the following, there is a list of some commands that come in handy when working with notebooks

- Starting a line with the `!` symbol will execute the following command in bash, e.g. `!pwd`.
- A function, object, ... directly followed by a question mark (don't give any arguments or type parentheses) brings up its docstring, e.g. `list?`
- To reset your memory footprint, it may sometimes be useful to restart your kernel.
  For this go to `Kernel > Restart Kernel...` and confirm the `restart`.
  This will reset your whole notebook such that you will have to execute all required code again (including imports).